In [2]:
import warnings

from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score
from bayes_opt import BayesianOptimization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier

from utils import preprocess_data
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'tensorflow.keras'

In [ ]:
warnings.filterwarnings("ignore")

X_train, X_test, y_train, y_test, train_df, test_df = preprocess_data(standardise=True)

In [36]:
def create_cnn_model(learning_rate, dropout_rate, num_filters, kernel_size):
    model = Sequential()
    model.add(Conv1D(filters=int(num_filters), kernel_size=int(kernel_size), activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(Flatten())
    model.add(Dropout(rate=dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [37]:
def optimize_cnn(learning_rate, dropout_rate, num_filters, kernel_size):
    model = KerasClassifier(build_fn=create_cnn_model,
                            learning_rate=learning_rate,
                            dropout_rate=dropout_rate,
                            num_filters=num_filters,
                            kernel_size=kernel_size,
                            epochs=10,
                            batch_size=10,
                            verbose=0)

    return cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

In [38]:
param_space = {
    'learning_rate': (0.0001, 0.01),
    'dropout_rate': (0.1, 0.5),
    'num_filters': (10, 50),
    'kernel_size': (2, 5)
}

In [39]:
optimizer = BayesianOptimization(
    f=optimize_cnn,
    pbounds=param_space,
    random_state=42
)

optimizer.maximize(init_points=5, n_iter=25)

|   iter    |  target   | dropou... | kernel... | learni... | num_fi... |
-------------------------------------------------------------------------
| 1         | 0.8004    | 0.2498    | 4.852     | 0.007347  | 33.95     |
| 2         | 0.8167    | 0.1624    | 2.468     | 0.000675  | 44.65     |
| 3         | 0.8106    | 0.3404    | 4.124     | 0.0003038 | 48.8      |
| 4         | 0.8167    | 0.433     | 2.637     | 0.0019    | 17.34     |
| 5         | 0.7963    | 0.2217    | 3.574     | 0.004376  | 21.65     |
| 6         | 0.8004    | 0.1399    | 2.421     | 0.007504  | 44.6      |
| 7         | 0.8065    | 0.4739    | 4.476     | 0.004009  | 28.34     |
| 8         | 0.8106    | 0.4862    | 3.428     | 0.006884  | 48.29     |
| 9         | 0.7943    | 0.3083    | 3.659     | 0.009978  | 23.37     |
| 10        | 0.8065    | 0.3276    | 3.089     | 0.00459   | 11.04     |
| 11        | 0.8004    | 0.4748    | 4.252     | 0.002969  | 44.79     |
| 12        | 0.8024    | 0.1998    | 

In [42]:
best_params = optimizer.max['params']
best_model = create_cnn_model(
    learning_rate=best_params['learning_rate'],
    dropout_rate=best_params['dropout_rate'],
    num_filters=best_params['num_filters'],
    kernel_size=best_params['kernel_size']
)

best_model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0)
y_pred = best_model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to class labels

accuracy = accuracy_score(y_test, y_pred)
print(f"Optimized CNN Accuracy: {accuracy}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Optimized CNN Accuracy: 0.7804878048780488
